# SARSA 이해하기: 완벽 가이드

## 목차

- [SARSA란 무엇인가?](#what-is-sarsa)
- [SARSA 사용처 및 방법](#where-and-how-sarsa-is-used)
- [SARSA의 수학적 기초](#mathematical-foundation-of-sarsa)
  - [복잡한 원본 버전](#complex-original-version)
  - [단순화된 버전](#simplified-version)
- [SARSA 단계별 설명](#step-by-step-explanation-of-sarsa)
- [SARSA의 주요 구성 요소](#key-components-of-sarsa)
  - [Q-테이블](#q-table)
  - [탐험 대 활용](#exploration-vs-exploitation)
  - [학습률 (α)](#learning-rate-α)
  - [할인율 (γ)](#discount-factor-γ)
- [SARSA 대 Q-러닝](#sarsa-vs-q-learning)
- [실용 예제: 그리드 월드](#practical-example-grid-world)
  - [환경 설정](#setting-up-the-environment)
  - [간단한 환경 생성](#creating-a-simple-environment)
  - [SARSA 알고리즘 구현](#implementing-the-sarsa-algorithm)
  - [탐험 대 활용 전략](#exploration-vs-exploitation-strategy)
  - [SARSA 알고리즘 실행](#running-the-sarsa-algorithm)
  - [학습 과정 시각화](#visualizing-the-learning-process)
  - [Q-값 및 최적 정책 분석](#analyzing-q-values-and-optimal-policy)
  - [다른 하이퍼파라미터로 테스트 (선택 사항)](#testing-with-different-hyperparameters-optional)
- [다른 환경에 SARSA 적용 (절벽 보행)](#applying-sarsa-to-different-environments-cliff-walking)
- [일반적인 문제점 및 해결책](#common-challenges-and-solutions)
- [SARSA 대 다른 강화 학습 알고리즘](#sarsa-vs-other-reinforcement-learning-algorithms)
  - [SARSA의 장점](#advantages-of-sarsa)
  - [SARSA의 한계점](#limitations-of-sarsa)
  - [관련 알고리즘](#related-algorithms)
- [결론](#conclusion)

## SARSA란 무엇인가?

SARSA (State-Action-Reward-State-Action)는 마르코프 결정 과정 정책을 학습하는 데 사용되는 강화 학습 알고리즘입니다. 이는 모델-프리, 가치 기반, **온-폴리시(on-policy)** 학습 알고리즘입니다. 오프-폴리시(off-policy)인 Q-러닝과 달리, SARSA는 현재 따르고 있는 정책의 가치를 학습합니다.

SARSA라는 이름은 단일 업데이트에 필요한 정보의 순서를 반영합니다: 현재 상태 (S), 취한 행동 (A), 받은 보상 (R), 다음 상태 (S'), 그리고 다음 상태에서 현재 정책에 따라 취한 *다음 행동* (A').

## SARSA 사용처 및 방법

SARSA는 Q-러닝과 유사한 시나리오에서 사용되지만, 온-폴리시 특성 때문에 현재 실행 중인 정책을 평가하거나 개선하는 것이 중요하거나 학습 중 안전이 우려되는 상황에 특히 적합합니다. 응용 분야는 다음과 같습니다:

1.  **로보틱스**: 학습 중에 일관성 있고 (잠재적으로 더 안전한) 정책을 따르는 것이 선호되는 탐색 또는 제어 작업 학습.
2.  **제어 시스템**: 취한 행동이 시스템의 상태와 후속 학습에 직접적인 영향을 미치는 컨트롤러 최적화 (예: HVAC 제어).
3.  **게임 플레이**: 다른 에이전트의 행동 (또는 환경의 확률성)이 에이전트의 현재 정책에 따라 달라지는 환경에서 에이전트 훈련.
4.  **학습 안정성과 *현재* 행동의 가치를 이해하는 것이 중요한 모든 상황.**

SARSA는 다음과 같은 환경에서 잘 작동합니다:
- 환경에 이산적인 상태와 행동이 있는 경우.
- 온-폴리시 접근 방식이 필요한 경우 (실제로 취한 행동에 기반한 학습).
- 환경이 완전히 관찰 가능한 경우.

## SARSA의 수학적 기초

### 복잡한 원본 버전

SARSA 알고리즘은 다음 업데이트 규칙을 사용하여 Q-값을 업데이트합니다:

$$Q(s_t, a_t) \leftarrow Q(s_t, a_t) + \alpha \left[r_t + \gamma Q(s_{t+1}, a_{t+1}) - Q(s_t, a_t)\right]$$

여기서:
- $Q(s_t, a_t)$는 상태 $s_t$와 행동 $a_t$에 대한 Q-값입니다.
- $\alpha$는 학습률 (0 < $\alpha$ ≤ 1)입니다.
- $r_t$는 상태 $s_t$에서 행동 $a_t$를 취한 후 받은 보상입니다.
- $\gamma$는 미래 보상에 대한 할인율 (0 ≤ $\gamma$ ≤ 1)입니다.
- $s_{t+1}$은 행동 $a_t$를 취한 후 관찰된 다음 상태입니다.
- $a_{t+1}$은 상태 $s_{t+1}$에서 **현재 정책** (예: 현재 Q-값에 기반한 엡실론-그리디)을 사용하여 선택된 **다음 행동**입니다.
- 대괄호 안의 항 $[r_t + \gamma Q(s_{t+1}, a_{t+1}) - Q(s_t, a_t)]$는 다음 단계에 대해 실제로 선택된 행동에 기반한 시간차(Temporal Difference, TD) 오류입니다.

### 단순화된 버전

더 간단한 용어로, SARSA 업데이트는 다음과 같이 이해할 수 있습니다:

$$
Q_{\text{new}} = Q_{\text{old}} + \alpha \left[ R + \gamma Q_{\text{next}} - Q_{\text{old}} \right]
$$

여기서 $Q_{\text{next}}$는 정책에 따라 다음에 취해질 *특정 상태-행동 쌍* $(s', a')$의 Q-값입니다.

또는 훨씬 더 간단하게:

$$
Q_{\text{new}} = Q_{\text{old}} + \alpha \left[ \text{Target} - Q_{\text{old}} \right]
$$

여기서 "Target"은 보상에 정책에 의해 선택된 *다음 상태-행동 쌍*의 할인된 가치를 더한 값입니다.

## SARSA 단계별 설명

1.  **Q-테이블 초기화**: 각 상태에 대한 행과 각 행동에 대한 열을 가진 테이블을 만들고, 처음에는 0이나 작은 무작위 값으로 채웁니다.
2.  **에피소드 시작**: 초기 상태 $s$에서 시작합니다.
3.  **행동 선택**: 상태 $s$에서 현재 Q-값을 기반으로 탐험/활용 정책(예: 엡실론-그리디)을 사용하여 행동 $a$를 선택합니다.
4.  **에피소드의 각 단계에 대해 반복**:
    a.  **행동 수행**: 행동 $a$를 실행하고, 보상 $r$과 새로운 상태 $s'$를 관찰합니다.
    b.  **다음 행동 선택**: 새로운 상태 $s'$에서 상태 $s'$에 대한 현재 Q-값을 기반으로 동일한 정책(예: 엡실론-그리디)을 사용하여 *다음 행동* $a'$를 선택합니다.
    c.  **Q-값 업데이트**: $s, a, r, s', a'$를 사용하여 SARSA 업데이트 공식을 적용합니다.
    d.  **상태 및 행동 업데이트**: $s \leftarrow s'$ 및 $a \leftarrow a'$로 설정합니다.
    e.  **종료 확인**: $s$가 종료 상태이면 에피소드를 종료합니다.
5.  **여러 에피소드 반복**: 에이전트가 따르는 정책에 따라 Q-값을 개선할 수 있도록 많은 에피소드를 실행합니다.

## SARSA의 주요 구성 요소

### Q-테이블
Q-테이블은 다음과 같은 조회 테이블입니다:
- 행은 환경의 상태를 나타냅니다.
- 열은 가능한 행동을 나타냅니다.
- 각 셀에는 해당 상태에서 해당 행동을 취하고 *그 이후 현재 정책을 따를 경우* 예상되는 미래 보상을 나타내는 Q-값이 포함됩니다.

예를 들어, 간단한 그리드 월드에서:

| 상태  | 위  | 아래 | 왼쪽 | 오른쪽 |
|-------|-----|------|------|-------|
| (0,0) | 0.0 | 0.0  | 0.0  | 0.0   |
| (0,1) | 0.0 | 0.0  | 0.0  | 0.0   |
| ...   | ... | ...  | ...  | ...   |

### 탐험 대 활용
탐험(새로운 행동 시도)과 활용(알려진 좋은 행동 사용) 사이의 균형은 중요합니다. SARSA는 Q-러닝과 마찬가지로 일반적으로 **엡실론-그리디** 전략을 사용합니다:
- 확률 $\epsilon$으로 무작위 행동을 선택합니다 (탐험).
- 확률 $1-\epsilon$으로 가장 높은 Q-값을 가진 행동을 선택합니다 (활용).
- $\epsilon$은 일반적으로 시간이 지남에 따라 감소합니다 (엡실론 감쇠).

### 학습률 (α)
- Q-값 업데이트에서 새로운 정보가 이전 정보를 얼마나 대체할지 제어합니다.
- 높은 $\alpha$ (1에 가까움): 빠르게 학습하지만 불안정할 수 있습니다.
- 낮은 $\alpha$ (0에 가까움): 느리게 학습하며 더 안정적인 수렴을 보입니다.
- 일반적인 값: 0.01 ~ 0.5.

### 할인율 (γ)
- 즉각적인 보상 대비 미래 보상의 중요성을 결정합니다.
- $\gamma = 0$: 즉각적인 보상만 고려합니다 (근시안적).
- $\gamma = 1$: 미래 보상을 즉각적인 보상과 동일하게 평가합니다 (원시안적, 보상이 멈추지 않으면 수렴하지 않을 수 있음).
- 일반적인 값: 0.9 ~ 0.99.

## SARSA 대 Q-러닝

주요 차이점은 Q-업데이트를 위한 목표값을 계산하는 방식에 있습니다:

-   **Q-러닝 (오프-폴리시)**: 다음 상태에서 가능한 최대 Q-값($max_{a'} Q(s', a')$)을 사용합니다. 탐험 중에 따르는 정책과 관계없이 최적의 정책을 학습합니다.
    $$Q(s, a) \leftarrow Q(s, a) + \alpha [r + \gamma \max_{a'} Q(s', a') - Q(s, a)]$$
-   **SARSA (온-폴리시)**: 현재 정책에 의해 선택된 *실제 다음 행동*의 Q-값($Q(s', a')$)을 사용합니다. 현재 실행 중인 정책(탐험 단계 포함)의 가치를 학습합니다.
    $$Q(s, a) \leftarrow Q(s, a) + \alpha [r + \gamma Q(s', a') - Q(s, a)]$$

이로 인해 SARSA는 일반적으로 더 보수적이며, 특히 위험(예: 절벽)이 있는 환경에서 탐험적 이동의 잠재적인 부정적 결과를 고려하기 때문입니다. Q-러닝은 최적의 경로를 직접 목표로 하며, 때로는 결국 더 높은 보상을 약속한다면 더 위험한 정책을 학습하기도 합니다.

## 실용 예제: 그리드 월드

다음 섹션에서는 Q-러닝 예제에서 사용된 동일한 그리드 월드 환경에 SARSA를 적용할 것입니다:
- 4×4 그리드.
- (0,0)에 보상 1, (3,3)에 보상 10인 종료 상태.
- 다른 모든 상태는 보상 0.
- 에이전트는 위, 아래, 왼쪽, 오른쪽으로 이동할 수 있습니다.
- 목표는 가장 높은 보상 상태에 도달하는 정책을 학습하는 것입니다.

알고리즘을 구현하고, 훈련을 실행하고, 학습된 Q-값과 정책을 시각화하여 Q-러닝이 찾을 수 있는 것과 암묵적으로 비교할 것입니다.

# 환경 설정
수치 연산을 위한 NumPy와 시각화를 위한 Matplotlib을 포함한 필요한 라이브러리를 가져옵니다.

In [ ]:
# 필요한 라이브러리 가져오기
import numpy as np  # 수치 연산을 위한 라이브러리
import matplotlib.pyplot as plt  # 시각화를 위한 라이브러리

# 타입 힌트 가져오기
from typing import List, Tuple, Dict, Optional

# 재현성을 위한 시드 설정
np.random.seed(42)

# Jupyter Notebook에서 인라인 플로팅 활성화
%matplotlib inline

# 간단한 환경 생성

SARSA 알고리즘을 위한 간단한 환경을 만들기 위해 4x4 그리드 월드를 정의할 것입니다. 그리드 월드는 다음과 같은 속성을 가집니다:

- 4개의 행과 4개의 열
- 가능한 행동: 'up', 'down', 'left', 'right'
- 보상이 있는 특정 종료 상태.
- 절벽 상태 (나중에 절벽 보행 예제에서 사용).


In [ ]:
# 그리드 월드 환경 생성 함수 정의
def create_gridworld(
    rows: int,
    cols: int,
    terminal_states: List[Tuple[int, int]],
    rewards: Dict[Tuple[int, int], int]
) -> Tuple[np.ndarray, List[Tuple[int, int]], List[str]]:
    """
    간단한 그리드 월드 환경을 생성합니다.

    매개변수:
    - rows (int): 그리드의 행 수.
    - cols (int): 그리드의 열 수.
    - terminal_states (List[Tuple[int, int]]): (행, 열) 튜플로 구성된 종료 상태 목록.
    - rewards (Dict[Tuple[int, int], int]): (행, 열)을 보상 값에 매핑하는 딕셔너리.

    반환값:
    - grid (np.ndarray): 보상을 나타내는 2D 배열 (참고용, 에이전트가 사용하지 않음).
    - state_space (List[Tuple[int, int]]): 그리드의 모든 가능한 상태 목록.
    - action_space (List[str]): 가능한 행동 목록 ('up', 'down', 'left', 'right').
    """
    # 그리드를 0으로 초기화 (시각화/참고용)
    grid = np.zeros((rows, cols))

    # 지정된 상태에 보상 할당
    for (row, col), reward in rewards.items():
        grid[row, col] = reward

    # 상태 공간을 가능한 모든 (행, 열) 쌍으로 정의
    state_space = [
        (row, col)
        for row in range(rows)
        for col in range(cols)
    ]

    # 행동 공간을 네 가지 가능한 이동으로 정의
    action_space = ['up', 'down', 'left', 'right']

    return grid, state_space, action_space

다음으로 상태 전이 함수가 필요합니다. 이 함수는 현재 상태와 행동을 입력으로 받아 다음 상태를 반환합니다. 이는 에이전트가 취하는 행동에 따라 그리드 주위를 이동하는 것으로 생각할 수 있습니다. 이 경우 환경은 결정론적입니다.

In [ ]:
# 상태 전이 함수 정의
def state_transition(state: Tuple[int, int], action: str, rows: int, cols: int) -> Tuple[int, int]:
    """
    현재 상태와 행동이 주어졌을 때 다음 상태를 계산합니다. 경계를 처리합니다.

    매개변수:
    - state (Tuple[int, int]): 현재 상태 (행, 열).
    - action (str): 취할 행동 ('up', 'down', 'left', 'right').
    - rows (int): 그리드의 행 수.
    - cols (int): 그리드의 열 수.

    반환값:
    - Tuple[int, int]: 행동을 취한 후의 결과 상태 (행, 열).
    """
    # 현재 상태를 행과 열로 분해
    row, col = state
    next_row, next_col = row, col

    # 행동에 따라 행 또는 열을 업데이트하고 경계를 존중하도록 보장
    if action == 'up' and row > 0:  # 가장 위쪽 행이 아니면 위로 이동
        next_row -= 1
    elif action == 'down' and row < rows - 1:  # 가장 아래쪽 행이 아니면 아래로 이동
        next_row += 1
    elif action == 'left' and col > 0:  # 가장 왼쪽 열이 아니면 왼쪽으로 이동
        next_col -= 1
    elif action == 'right' and col < cols - 1:  # 가장 오른쪽 열이 아니면 오른쪽으로 이동
        next_col += 1
    # 행동이 그리드 밖으로 이동하게 하면 같은 상태에 머무름 (행, 열 변경 없음)

    # 새로운 상태를 튜플로 반환
    return (next_row, next_col)

이제 에이전트가 환경과 상호 작용할 수 있으므로 보상 함수를 정의해야 합니다. 이 함수는 주어진 상태에 도달했을 때의 보상을 반환하며, 이는 훈련 중 Q-값을 업데이트하는 데 사용됩니다.

In [ ]:
# 보상 함수 정의
def get_reward(state: Tuple[int, int], rewards: Dict[Tuple[int, int], int]) -> int:
    """
    주어진 상태에 대한 보상을 가져옵니다.

    매개변수:
    - state (Tuple[int, int]): 현재 상태 (행, 열).
    - rewards (Dict[Tuple[int, int], int]): 상태 (행, 열)를 보상 값에 매핑하는 딕셔너리.

    반환값:
    - int: 주어진 상태에 대한 보상. 상태가 보상 딕셔너리에 없으면 0을 반환합니다.
    """
    # 보상 딕셔너리를 사용하여 주어진 상태에 대한 보상을 가져옵니다.
    # 상태를 찾을 수 없으면 기본 보상 0을 반환합니다.
    return rewards.get(state, 0)

이제 그리드 월드 환경과 필요한 헬퍼 함수를 정의했으므로 간단한 예제로 테스트해 봅시다. (0, 0)과 (3, 3)에 각각 1과 10의 보상을 갖는 두 개의 종료 상태가 있는 4x4 그리드를 생성합니다. 그런 다음 상태 (2, 2)에서 위쪽으로 이동하여 상태 전이 및 보상 함수를 테스트합니다.

In [ ]:
# 그리드 월드 환경 예제 사용법

# 그리드 차원 (4x4), 종료 상태 및 보상 정의
rows, cols = 4, 4  # 그리드의 행과 열 수
terminal_states = [(0, 0), (3, 3)]  # 종료 상태
rewards = {(0, 0): 1, (3, 3): 10}  # 종료 상태에 대한 보상 (다른 상태는 보상 0)

# 그리드 월드 환경 생성
grid, state_space, action_space = create_gridworld(rows, cols, terminal_states, rewards)

# 상태 전이 및 보상 함수 테스트
current_state = (2, 2)  # 시작 상태
action = 'up'  # 취할 행동
next_state = state_transition(current_state, action, rows, cols)  # 다음 상태 계산
reward = get_reward(next_state, rewards)  # 다음 상태에 대한 보상 가져오기

# 결과 출력
print("GridWorld (rewards view):")  # 보상이 있는 그리드 표시
print(grid)
print(f"Current State: {current_state}")  # 현재 상태 표시
print(f"Action Taken: {action}")  # 취한 행동 표시
print(f"Next State: {next_state}")  # 결과 다음 상태 표시
print(f"Reward at Next State: {reward}")  # 다음 상태에 대한 보상 표시

지정된 차원, 종료 상태 및 보상으로 그리드 월드 환경이 생성된 것을 볼 수 있습니다. 시작 상태 (2, 2)와 행동 ('up')을 선택했습니다. 다음 상태는 (1, 2)로 계산되며, 상태 (1, 2)에 도달하는 보상은 지정된 보상 상태가 아니므로 0입니다.

# SARSA 알고리즘 구현

그리드 월드 환경을 성공적으로 구현했습니다. 이제 SARSA 알고리즘을 구현합니다. 먼저 상태-행동 쌍을 Q-값(예상 누적 보상)에 매핑하는 Q-테이블을 초기화합니다.

In [ ]:
# Q-테이블 초기화
def initialize_q_table(state_space: List[Tuple[int, int]], action_space: List[str]) -> Dict[Tuple[Tuple[int, int], str], float]:
    """
    모든 상태-행동 쌍에 대해 Q-테이블을 0으로 초기화합니다.
    키로 (상태, 행동) 튜플을 사용하는 단일 딕셔너리를 사용합니다.

    매개변수:
    - state_space (List[Tuple[int, int]]): 모든 가능한 상태 목록.
    - action_space (List[str]): 모든 가능한 행동 목록.

    반환값:
    - q_table (Dict[Tuple[Tuple[int, int], str], float]): (상태, 행동) 쌍을 Q-값에 매핑하는 딕셔너리, 0.0으로 초기화됨.
    """
    q_table: Dict[Tuple[Tuple[int, int], str], float] = {}
    for state in state_space:
        for action in action_space:
            # (상태, 행동) 쌍에 대한 Q-값을 0.0으로 초기화
            q_table[(state, action)] = 0.0
    return q_table

# --- 대안 Q-테이블 구조 (중첩 딕셔너리) ---
# 쉬운 비교 및 재사용을 위해. run_sarsa_episode가 적응할 것입니다.
def initialize_q_table_nested(state_space: List[Tuple[int, int]], action_space: List[str]) -> Dict[Tuple[int, int], Dict[str, float]]:
    """
    중첩 딕셔너리 구조를 사용하여 Q-테이블을 0으로 초기화합니다.

    매개변수:
    - state_space (List[Tuple[int, int]]): 모든 가능한 상태 목록.
    - action_space (List[str]): 모든 가능한 행동 목록.

    반환값:
    - q_table (Dict[Tuple[int, int], Dict[str, float]]): q_table[state][action]이 Q-값을 제공하는 중첩 딕셔너리.
    """
    q_table: Dict[Tuple[int, int], Dict[str, float]] = {}
    for state in state_space:
        # 종료 상태의 경우, 해당 상태에서 취할 수 있는 행동이 없으므로 Q-값은 0으로 유지되어야 합니다.
        # 그러나 모두 초기화하면 종료 확인 전에 업데이트 중 조회가 더 쉬워집니다.
        q_table[state] = {action: 0.0 for action in action_space}
    return q_table

다음으로, 행동 선택을 위한 엡실론-그리디 정책을 정의합니다. 무작위 값이 엡실론보다 작으면 탐험(무작위 행동 선택)하고, 그렇지 않으면 활용(가장 잘 알려진 행동 선택)합니다.

In [ ]:
# 엡실론-그리디 정책을 사용하여 행동 선택 (중첩 Q-테이블 구조 사용)
def epsilon_greedy_policy(
    state: Tuple[int, int],
    q_table: Dict[Tuple[int, int], Dict[str, float]],
    action_space: List[str],
    epsilon: float
) -> str:
    """
    중첩 Q-테이블에서 엡실론-그리디 정책을 사용하여 행동을 선택합니다.

    매개변수:
    - state (Tuple[int, int]): 현재 상태 (행, 열).
    - q_table (Dict[Tuple[int, int], Dict[str, float]]): 상태 -> 행동 -> Q-값을 매핑하는 중첩 Q-테이블.
    - action_space (List[str]): 가능한 행동 목록.
    - epsilon (float): 탐험률 (0 <= epsilon <= 1).

    반환값:
    - str: 선택된 행동.
    """
    # 상태가 유효하고 Q-테이블에 존재하는지 확인
    if state not in q_table:
        # 필요한 경우 보지 못한 상태 처리, 예: 무작위 행동 반환
        return np.random.choice(action_space)

    # 확률 엡실론으로 무작위 행동 선택 (탐험)
    if np.random.rand() < epsilon:
        return np.random.choice(action_space)
    # 그렇지 않으면 현재 상태에 대해 가장 높은 Q-값을 가진 행동 선택 (활용)
    else:
        # 최댓값을 찾기 전에 상태가 존재하고 행동을 가지고 있는지 확인
        if q_table[state]:
            return max(q_table[state], key=q_table[state].get)
        else:
            # 상태에 행동이 없는 경우 (예: 희소 초기화 시 발생 가능), 무작위로 선택
            return np.random.choice(action_space)

상태 $s$에서 행동 $a$를 취하고 보상 $r$과 다음 상태 $s'$를 관찰한 후, 상태 $s'$에서 정책을 사용하여 *다음 행동* $a'$를 선택합니다. 그런 다음 SARSA 업데이트 규칙을 사용하여 $(s, a)$에 대한 Q-값을 업데이트합니다:

$$
Q(s, a) \leftarrow Q(s, a) + \alpha \left[ r + \gamma Q(s', a') - Q(s, a) \right]
$$

In [ ]:
# SARSA 규칙을 사용하여 Q-값 업데이트 (중첩 Q-테이블 구조 사용)
def update_sarsa_value(
    q_table: Dict[Tuple[int, int], Dict[str, float]],
    state: Tuple[int, int],
    action: str,
    reward: int,
    next_state: Tuple[int, int],
    next_action: str, # 다음 상태에서 정책에 의해 선택된 행동
    alpha: float,
    gamma: float
) -> None:
    """
    중첩 Q-테이블을 사용하여 SARSA 업데이트 규칙으로 Q-값을 업데이트합니다.

    매개변수:
    - q_table (Dict[Tuple[int, int], Dict[str, float]]): 중첩 Q-테이블.
    - state (Tuple[int, int]): 현재 상태 (s).
    - action (str): 취한 행동 (a).
    - reward (int): 받은 보상 (r).
    - next_state (Tuple[int, int]): 관찰된 다음 상태 (s').
    - next_action (str): 정책에 의해 다음 상태에서 선택된 행동 (a').
    - alpha (float): 학습률.
    - gamma (float): 할인율.

    반환값:
    - None: Q-테이블을 제자리에서 업데이트합니다.
    """
    # KeyError를 피하기 위해 상태와 행동이 Q-테이블에 존재하는지 확인
    if state not in q_table or action not in q_table[state]:
        # 선택적으로 상태/행동 쌍이 새로운 경우 초기화 (전체 초기화에서는 발생하지 않아야 함)
        # 또는 경고/오류 로깅
        return
    if next_state not in q_table or next_action not in q_table[next_state]:
         # 다음 상태가 종료 상태이거나 행동이 유효하지 않으면 Q(s', a')는 종종 0으로 처리됩니다.
         # 또는 초기화가 완전하지 않은 경우 보지 못한 상태를 적절히 처리합니다.
         q_next = 0.0
    else:
        # 다음 상태와 *실제로* 선택된 다음 행동에 대한 Q-값을 가져옵니다.
        q_next: float = q_table[next_state][next_action]

    # TD 목표 계산: R + gamma * Q(s', a')
    td_target: float = reward + gamma * q_next

    # TD 오류 계산: TD 목표 - Q(s, a)
    td_error: float = td_target - q_table[state][action]

    # SARSA 공식을 사용하여 현재 상태-행동 쌍에 대한 Q-값 업데이트
    q_table[state][action] += alpha * td_error

지금까지 환경과 핵심 SARSA 업데이트 로직을 정의했습니다. 이제 이를 SARSA의 단일 에피소드를 실행하는 함수로 결합합니다. 여기에는 환경을 단계별로 진행하고, 행동을 선택하고, 종료 상태에 도달할 때까지 Q-값을 업데이트하는 과정이 포함됩니다.

In [ ]:
# SARSA의 단일 에피소드 실행
def run_sarsa_episode(
    q_table: Dict[Tuple[int, int], Dict[str, float]],
    state_space: List[Tuple[int, int]],
    action_space: List[str],
    rewards: Dict[Tuple[int, int], int],
    terminal_states: List[Tuple[int, int]],
    rows: int,
    cols: int,
    alpha: float,
    gamma: float,
    epsilon: float,
    max_steps: int
) -> Tuple[int, int]:
    """
    SARSA의 단일 에피소드를 실행합니다.

    매개변수:
    - q_table (Dict[Tuple[int, int], Dict[str, float]]): 중첩 Q-테이블.
    - state_space (List[Tuple[int, int]]): 모든 가능한 상태 목록.
    - action_space (List[str]): 가능한 행동 목록.
    - rewards (Dict[Tuple[int, int], int]): 상태를 보상에 매핑하는 딕셔너리.
    - terminal_states (List[Tuple[int, int]]): 종료 상태 목록.
    - rows (int): 그리드의 행 수.
    - cols (int): 그리드의 열 수.
    - alpha (float): 학습률.
    - gamma (float): 할인율.
    - epsilon (float): 탐험률.
    - max_steps (int): 에피소드당 최대 단계 수.

    반환값:
    - total_reward (int): 에피소드 동안 누적된 총 보상.
    - steps (int): 에피소드에서 수행된 단계 수.
    """
    # 필요한 경우 무작위 비종료 상태에서 시작하거나 고정 시작 상태에서 시작
    # 여기서는 더 넓은 초기 탐험을 위해 무작위 상태에서 시작
    state: Tuple[int, int] = state_space[np.random.choice(len(state_space))]
    while state in terminal_states: # 시작 상태가 종료 상태가 아닌지 확인
        state = state_space[np.random.choice(len(state_space))]

    # 시작 상태 's'에서 정책을 사용하여 첫 번째 행동 'a' 선택
    action: str = epsilon_greedy_policy(state, q_table, action_space, epsilon)

    total_reward: int = 0  # 에피소드의 총 보상 초기화
    steps: int = 0  # 단계 카운터 초기화

    # 최대 단계 수 동안 또는 종료 상태에 도달할 때까지 반복
    for _ in range(max_steps):
        # 행동 'a'를 취하고 다음 상태 's''와 보상 'r' 관찰
        next_state: Tuple[int, int] = state_transition(state, action, rows, cols)
        reward: int = get_reward(next_state, rewards)
        total_reward += reward # 보상 누적

        # 정책을 사용하여 *다음* 상태 's''에서 *다음* 행동 'a'' 선택
        next_action: str = epsilon_greedy_policy(next_state, q_table, action_space, epsilon)

        # SARSA 규칙을 사용하여 상태-행동 쌍 (s, a)에 대한 Q-값 업데이트
        # 참고: Q-값은 s, a, r, s', a'를 기반으로 업데이트됨
        update_sarsa_value(q_table, state, action, reward, next_state, next_action, alpha, gamma)

        # 다음 반복을 위해 현재 상태와 행동 업데이트
        state = next_state
        action = next_action

        steps += 1

        # 에이전트가 종료 상태에 도달했는지 확인
        if state in terminal_states:
            break # 종료 상태에 도달하면 에피소드 종료

    # 이 에피소드의 총 보상과 단계 수 반환
    return total_reward, steps

# 탐험 대 활용 전략

탐험과 활용의 균형을 적절히 맞추기 위해 이전에 정의한 엡실론-그리디 정책을 사용하고 동적 엡실론 조정을 구현합니다. 엡실론은 높은 값(더 많은 탐험)에서 시작하여 점차 감소하여 에이전트가 학습함에 따라 더 많은 활용을 장려합니다.

In [ ]:
# 동적 엡실론 조정 함수 정의 (Q-러닝 참조와 동일)
def adjust_epsilon(
    initial_epsilon: float,
    min_epsilon: float,
    decay_rate: float,
    episode: int
) -> float:
    """
    지수 감쇠를 사용하여 시간이 지남에 따라 엡실론을 동적으로 조정합니다.

    매개변수:
    - initial_epsilon (float): 초기 탐험률.
    - min_epsilon (float): 최소 탐험률.
    - decay_rate (float): 엡실론이 감쇠하는 비율.
    - episode (int): 현재 에피소드 번호.

    반환값:
    - float: 현재 에피소드에 대한 조정된 탐험률.
    """
    # 감쇠된 엡실론 값을 계산하고 최소 엡실론 아래로 내려가지 않도록 보장
    return max(min_epsilon, initial_epsilon * np.exp(-decay_rate * episode))

계획된 에피소드에 걸쳐 엡실론 감쇠를 추적하고 플로팅하여 탐험 전략을 시각화해 봅시다.

In [ ]:
# 동적 엡실론 조정의 예제 사용 및 감쇠 플로팅

# 엡실론 파라미터 정의
initial_epsilon: float = 1.0  # 완전 탐험으로 시작
min_epsilon: float = 0.1  # 최소 탐험률
decay_rate: float = 0.01  # 엡실론 감쇠율
episodes: int = 500  # 훈련을 위한 총 에피소드 수

# 에피소드에 걸쳐 엡실론 값 추적
epsilon_values: List[float] = []
for episode in range(episodes):
    # 현재 에피소드에 대한 엡실론 조정
    current_epsilon = adjust_epsilon(initial_epsilon, min_epsilon, decay_rate, episode)
    epsilon_values.append(current_epsilon)
    

# 에피소드에 걸쳐 엡실론 감쇠 플로팅
plt.figure(figsize=(20, 3)) # 그림 크기를 약간 조정
plt.plot(epsilon_values)
plt.xlabel('Episode')  # x축 레이블
plt.ylabel('Epsilon')  # y축 레이블
plt.title('Epsilon Decay Over Episodes')  # 플롯 제목
plt.grid(True) # 가독성을 위해 그리드 추가
plt.show()  # 플롯 표시

플롯은 엡실론이 1.0(순수 탐험)에서 시작하여 최소값 0.1(10% 탐험으로 대부분 활용)으로 지수적으로 감쇠하는 것을 보여줍니다. 이러한 점진적인 변화는 에이전트가 초기에 환경을 발견한 다음 학습한 내용을 기반으로 정책을 개선할 수 있도록 합니다.

# SARSA 알고리즘 실행

이제 그리드 월드 환경에서 동적 엡실론 조정을 사용하여 여러 에피소드에 걸쳐 SARSA 알고리즘을 실행합니다. 각 에피소드에 대한 총 보상과 에피소드 길이를 추적합니다.

In [ ]:
# 여러 에피소드에 걸쳐 SARSA를 실행하고 성능을 추적하는 함수
def run_sarsa(
    state_space: List[Tuple[int, int]],
    action_space: List[str],
    rewards: Dict[Tuple[int, int], int],
    terminal_states: List[Tuple[int, int]],
    rows: int,
    cols: int,
    alpha: float,
    gamma: float,
    initial_epsilon: float,
    min_epsilon: float,
    decay_rate: float,
    episodes: int,
    max_steps: int
) -> Tuple[Dict[Tuple[int, int], Dict[str, float]], List[int], List[int]]:
    """
    엡실론 감쇠를 사용하여 여러 에피소드에 걸쳐 SARSA 알고리즘을 실행합니다.

    매개변수:
    - state_space, action_space, rewards, terminal_states, rows, cols: 환경 파라미터.
    - alpha, gamma: SARSA 하이퍼파라미터.
    - initial_epsilon, min_epsilon, decay_rate: 엡실론 파라미터.
    - episodes: 실행할 에피소드 수.
    - max_steps: 에피소드당 최대 단계 수.

    반환값:
    - q_table: 학습된 Q-테이블 (중첩 딕셔너리).
    - rewards_per_episode: 에피소드별 총 보상 목록.
    - episode_lengths: 에피소드 길이 목록.
    """
    # Q-테이블 초기화 (헬퍼와의 호환성을 위해 중첩 구조 사용)
    q_table: Dict[Tuple[int, int], Dict[str, float]] = initialize_q_table_nested(state_space, action_space)

    # 메트릭을 저장할 목록 초기화
    rewards_per_episode: List[int] = []
    episode_lengths: List[int] = []

    # 각 에피소드 반복
    for episode in range(episodes):
        # 현재 에피소드에 대한 엡실론 조정
        epsilon: float = adjust_epsilon(initial_epsilon, min_epsilon, decay_rate, episode)

        # SARSA의 단일 에피소드 실행
        total_reward, steps = run_sarsa_episode(
            q_table, state_space, action_space, rewards, terminal_states,
            rows, cols, alpha, gamma, epsilon, max_steps
        )

        # 현재 에피소드의 메트릭 추가
        rewards_per_episode.append(total_reward)
        episode_lengths.append(steps)

    # 학습된 Q-테이블 및 메트릭 반환
    return q_table, rewards_per_episode, episode_lengths

하이퍼파라미터를 설정하고 SARSA 훈련 프로세스를 실행합니다.

In [ ]:
# SARSA 하이퍼파라미터 설정
alpha: float = 0.1          # 학습률
gamma: float = 0.9          # 할인율
initial_epsilon: float = 1.0  # 초기 탐험률
min_epsilon: float = 0.1      # 최소 탐험률
decay_rate: float = 0.01     # 엡실론 감쇠율
episodes: int = 500         # 에피소드 수
max_steps: int = 100        # 에피소드당 최대 단계 수

# 명확성을 위해 그리드 월드 환경 파라미터 다시 정의
rows, cols = 4, 4
terminal_states = [(0, 0), (3, 3)]
rewards = {(0, 0): 1, (3, 3): 10}
grid, state_space, action_space = create_gridworld(rows, cols, terminal_states, rewards)

# SARSA 알고리즘 실행
sarsa_q_table, sarsa_rewards_per_episode, sarsa_episode_lengths = run_sarsa(
    state_space, action_space, rewards, terminal_states, rows, cols, alpha, gamma,
    initial_epsilon, min_epsilon, decay_rate, episodes, max_steps
)

print("SARSA training completed.") # SARSA 훈련 완료.

# 학습 과정 시각화

에피소드별 총 보상과 각 에피소드의 길이를 플로팅하여 훈련 진행 상황을 시각화해 봅시다.

In [ ]:
# 더 나은 가시성을 위해 그림 크기 조정
plt.figure(figsize=(20, 3))

# 에피소드별 총 보상 플로팅
plt.subplot(1, 2, 1)
plt.plot(sarsa_rewards_per_episode)
plt.xlabel('Episode')
plt.ylabel('Total Reward')
plt.title('SARSA: Cumulative Rewards Over Episodes') # SARSA: 에피소드별 누적 보상
plt.grid(True)

# 에피소드별 에피소드 길이 플로팅
plt.subplot(1, 2, 2)
plt.plot(sarsa_episode_lengths)
plt.xlabel('Episode')
plt.ylabel('Episode Length')
plt.title('SARSA: Episode Lengths Over Episodes') # SARSA: 에피소드별 에피소드 길이
plt.grid(True)

# 레이아웃 조정 및 플롯 표시
plt.tight_layout()
plt.show()

**SARSA 학습 곡선 분석**

**왼쪽 그래프: 에피소드별 누적 보상**  
- 초기 에피소드에서는 **높은 탐험 (엡실론-그리디 정책)**으로 인해 보상이 크게 변동합니다.  
- 시간이 지남에 따라 에이전트는 **목표 상태 (보상 10)를 점점 더 많이 찾지만**, 패턴은 다소 불규칙하게 유지되어 지속적인 탐험을 나타냅니다.  
- **결정론적 Q-러닝과 달리 SARSA는 보상에 약간의 분산을 유지합니다.** 이는 각 단계에서 최상의 가능한 행동 대신 취해진 행동을 기반으로 학습하기 때문입니다.  
- 에이전트는 **결국 더 높은 보상을 달성하는 방향으로 안정화**되지만, 가끔 낮은 보상의 에피소드는 간헐적인 차선책 탐험을 시사합니다.  

**오른쪽 그래프: 에피소드별 에피소드 길이**  
- **초기 에피소드는 긴 에피소드 길이를 보여주며**, 비효율적이고 탐험적인 경로를 나타냅니다.  
- **학습이 진행됨에 따라 에피소드 길이는 상당히 감소**하며, 이는 에이전트가 더 짧고 최적의 경로를 찾고 있음을 시사합니다.  
- **후반 에피소드에서도 일부 변동이 남아 있으며**, 이는 탐험이 계속 의사 결정에 영향을 미치는 SARSA의 온-폴리시 특성 때문일 가능성이 높습니다.  
- 전반적인 **하향 추세는 성공적인 학습을 확인**시켜 주며, 에이전트가 일관되게 목표에 더 빨리 도달합니다.  

**전체 해석**  
- **SARSA 에이전트는 환경 탐색을 성공적으로 학습**하여 에피소드 길이를 줄이고 누적 보상을 향상시킵니다.  
- **Q-러닝과 비교할 때, SARSA는 정책 의존적 업데이트로 인해 학습 곡선에서 더 많은 변동성을 보입니다.**  
- 최종 정책은 **순전히 탐욕적이지 않으며**, SARSA는 탐험과 활용을 보다 자연스럽게 균형 맞춥니다.  
- 전반적인 **패턴은 예상되는 강화 학습 행동을 따릅니다**: 초기 불안정성, 점진적 개선, 그리고 결국 최적에 가까운 성능으로의 수렴.

# Q-값 및 최적 정책 분석

이제 SARSA 알고리즘에서 파생된 학습된 Q-값과 결과 정책을 시각화합니다. 각 행동에 대한 Q-값에는 히트맵을 사용하고 정책에는 그리드에 화살표를 사용합니다.

In [ ]:
# Q-값 히트맵 시각화 함수
def plot_q_values_heatmap(q_table: Dict[Tuple[int, int], Dict[str, float]], rows: int, cols: int, action_space: List[str], fig: plt.Figure, axes: np.ndarray) -> None:
    """
    제공된 축에 각 행동에 대한 Q-값을 히트맵으로 시각화합니다.

    매개변수:
    - q_table: 상태를 행동 및 해당 Q-값에 매핑하는 중첩 딕셔너리.
    - rows: 그리드의 행 수.
    - cols: 그리드의 열 수.
    - action_space: 가능한 행동 목록 (예: ['up', 'down', 'left', 'right']).
    - fig: Matplotlib 그림 객체.
    - axes: 히트맵 플로팅을 위한 Matplotlib 축 배열.
    """
    for i, action in enumerate(action_space):
        # Q-테이블에 없는 상태에 대해 -inf로 그리드 초기화
        q_values = np.full((rows, cols), -np.inf)
        for (row, col), actions in q_table.items():
            if action in actions:
                q_values[row, col] = actions[action]  # 행동에 대한 Q-값 할당

        # 현재 행동에 대한 히트맵 플로팅
        ax = axes[i]
        cax = ax.matshow(q_values, cmap='viridis')  # 'viridis' 컬러맵 사용
        fig.colorbar(cax, ax=ax)  # 히트맵에 컬러바 추가
        ax.set_title(f"SARSA Q-values: {action}")  # 히트맵 제목

        # 더 나은 시각화를 위해 그리드 라인 추가
        ax.set_xticks(np.arange(-.5, cols, 1), minor=True)
        ax.set_yticks(np.arange(-.5, rows, 1), minor=True)
        ax.grid(which='minor', color='w', linestyle='-', linewidth=1)

        # 깔끔한 모양을 위해 틱 레이블 제거
        ax.set_xticks(np.arange(cols))
        ax.set_yticks(np.arange(rows))
        ax.tick_params(axis='both', which='both', length=0)
        ax.set_xticklabels([])
        ax.set_yticklabels([])


# 학습된 정책 시각화 함수
def plot_policy_grid(q_table: Dict[Tuple[int, int], Dict[str, float]], rows: int, cols: int, terminal_states: List[Tuple[int, int]], ax: plt.Axes) -> None:
    """
    제공된 축에 학습된 정책을 그리드의 화살표로 시각화합니다.

    매개변수:
    - q_table: 상태를 행동 및 해당 Q-값에 매핑하는 중첩 딕셔너리.
    - rows: 그리드의 행 수.
    - cols: 그리드의 열 수.
    - terminal_states: 그리드의 종료 상태 목록.
    - ax: 정책 그리드 플로팅을 위한 Matplotlib 축.
    """
    # 정책 기호를 저장할 그리드 초기화
    policy_grid = np.empty((rows, cols), dtype=str)
    action_symbols = {'up': '↑', 'down': '↓', 'left': '←', 'right': '→', '': ''}

    for r in range(rows):
        for c in range(cols):
            state = (r, c)
            if state in terminal_states:
                # 종료 상태를 'T'로 표시
                policy_grid[r, c] = 'T'
                continue
            if state in q_table and q_table[state]:
                # 가장 높은 Q-값을 가진 행동 찾기
                max_q = -np.inf
                best_actions = []
                for action, q_val in q_table[state].items():
                    if q_val > max_q:
                        max_q = q_val
                        best_actions = [action]
                    elif q_val == max_q:
                        best_actions.append(action)

                # 동점일 경우 첫 번째 행동 선택
                if best_actions:
                    best_action = best_actions[0]
                    policy_grid[r, c] = action_symbols[best_action]
                else:
                    # 유효한 행동이 없는 상태 표시
                    policy_grid[r, c] = '.'
            else:
                # 방문하지 않았거나 Q-값이 없는 상태 표시
                policy_grid[r, c] = '.'

    # 정책 그리드 플로팅
    ax.matshow(np.zeros((rows, cols)), cmap='Greys', alpha=0.1)  # 배경 그리드
    for r in range(rows):
        for c in range(cols):
            # 각 셀에 정책 기호 추가
            ax.text(c, r, policy_grid[r, c], ha='center', va='center', fontsize=14, color='black' if policy_grid[r, c] != 'T' else 'red')

    # 그리드 라인 및 제목 추가
    ax.set_title("SARSA Learned Policy") # SARSA 학습된 정책
    ax.set_xticks(np.arange(-.5, cols, 1), minor=True)
    ax.set_yticks(np.arange(-.5, rows, 1), minor=True)
    ax.grid(which='minor', color='black', linestyle='-', linewidth=1)
    ax.set_xticks([])
    ax.set_yticks([])

In [ ]:
# Q-값 히트맵과 학습된 정책을 나란히 플로팅
fig, axes = plt.subplots(1, len(action_space) + 1, figsize=(20, 4))  # 히트맵 및 정책을 위한 서브플롯 생성

# 각 행동에 대한 Q-값 히트맵 플로팅
plot_q_values_heatmap(sarsa_q_table, rows, cols, action_space, fig, axes[:-1])

# 학습된 정책 플로팅
plot_policy_grid(sarsa_q_table, rows, cols, terminal_states, axes[-1])

# 레이아웃 조정 및 플롯 표시
plt.tight_layout()
plt.show()

**SARSA Q-값 및 정책 분석**

**Q-값 히트맵 (왼쪽 네 개 플롯)**  
- 이 히트맵들은 그리드의 모든 상태에서 각 특정 행동(위, 아래, 왼쪽, 오른쪽)을 취하는 것에 대한 학습된 Q-값을 보여줍니다.  
- **밝은 색상(노란색)**은 더 높은 Q-값을 나타내며, *현재 정책 하에서* 더 나은 장기 보상으로 이어질 것으로 추정되는 행동을 나타냅니다.  
- **아래쪽 및 오른쪽으로 이동하는 행동에 대해 더 높은 Q-값이 관찰되며**, 이는 학습된 정책의 목표 지향적 특성을 반영합니다.  
- 값은 목표 상태 (3,3)에서 바깥쪽으로 전파되어 보상이 그리드 전체의 학습에 어떻게 영향을 미치는지 보여줍니다.  

**학습된 정책 (가장 오른쪽 플롯)**  
- 이 그리드는 각 상태에 대해 가장 높은 Q-값을 가진 행동을 보여주며, 학습된 Q-값에서 파생된 **탐욕적 정책**을 나타냅니다. 'T'는 종료 상태를 표시합니다.  
- **화살표는 일반적으로 아래쪽(↓)과 오른쪽(→)을 가리키며**, 에이전트를 (3,3)의 목표로 안내합니다.  
- **왼쪽(←) 및 위쪽(↑) 행동은 탐험의 영향으로 인해 약간 더 높은 Q-값을 갖는 상태에서만 나타납니다.**  
- **정책 구조는 안정적이며**, 학습된 Q-값을 올바르게 반영합니다.  

**전체 해석:**  
- SARSA 에이전트는 **(엡실론-그리디) 정책을 따르는 것에 대한 기대 수익을 반영하는 Q-값을 성공적으로 학습했습니다.**  
- **파생된 탐욕적 정책은 목표 상태 (3,3)를 향한 명확한 경로를 보여주며**, 효율적인 아래-오른쪽 궤적을 따릅니다.  
- 가장 높은 기대 보상을 기반으로 업데이트하는 **Q-러닝과 비교할 때**, SARSA의 정책은 **탐험과 학습된 경험에 더 많은 영향을 받습니다.** 그러나 이 경우 최종 정책은 거의 동일하게 유지됩니다.  
- Q-값과 정책이 안정적으로 유지됨에 따라 **학습 과정이 수렴되었습니다.**

# Q-값 및 최적 정책 분석
SARSA가 학습한 최적 정책을 표 형식으로 살펴봅시다. 각 상태에서 각 행동에 대한 Q-값과 해당 최적 행동을 보여줍니다.

In [ ]:
# SARSA Q-테이블 데이터 및 파생 정책을 나타내는 딕셔너리 목록 생성
sarsa_q_policy_data = []
for r in range(rows):
    for c in range(cols):
        state = (r, c)  # 현재 상태를 튜플 (행, 열)로 정의
        if state in sarsa_q_table:  # 상태가 SARSA Q-테이블에 존재하는지 확인
            actions = sarsa_q_table[state]  # 현재 상태의 모든 행동에 대한 Q-값 검색
            if actions:  # 행동 딕셔너리가 비어 있지 않은지 확인
                # 가장 높은 Q-값을 기반으로 최적 행동 결정, 종료 상태의 경우 'Terminal'로 표시
                best_action = max(actions, key=actions.get) if state not in terminal_states else 'Terminal'
                # 정책 데이터 목록에 상태, Q-값 및 최적 행동 추가
                sarsa_q_policy_data.append({
                    'State': state,
                    'up': actions.get('up', 0.0),  # .get을 사용하여 'up'에 대한 Q-값을 안전하게 검색, 없으면 0.0으로 기본 설정
                    'down': actions.get('down', 0.0),  # 'down'에 대한 Q-값 검색
                    'left': actions.get('left', 0.0),  # 'left'에 대한 Q-값 검색
                    'right': actions.get('right', 0.0),  # 'right'에 대한 Q-값 검색
                    'Optimal Action': best_action  # 현재 상태의 최적 행동 저장
                })
            else:  # 상태에 대한 행동 딕셔너리가 비어 있는 경우 처리
                sarsa_q_policy_data.append({
                    'State': state,
                    'up': 0.0, 'down': 0.0, 'left': 0.0, 'right': 0.0,  # Q-값을 0.0으로 기본 설정
                    'Optimal Action': 'N/A'  # 최적 행동을 'N/A'로 표시
                })
        else:  # 상태가 Q-테이블에 없는 경우 처리 (전체 초기화에서는 발생하지 않아야 함)
            sarsa_q_policy_data.append({
                'State': state,
                'up': 0.0, 'down': 0.0, 'left': 0.0, 'right': 0.0,  # Q-값을 0.0으로 기본 설정
                'Optimal Action': 'N/A'  # 최적 행동을 'N/A'로 표시
            })

# 가독성을 위해 상태별로 정책 데이터 정렬 (선택 사항)
sarsa_q_policy_data.sort(key=lambda x: x['State'])

# Q-테이블 데이터를 표 형식으로 표시
header = ['State', 'up', 'down', 'left', 'right', 'Optimal Action']  # 테이블 헤더 정의
print(f"{header[0]:<10} {header[1]:<10} {header[2]:<10} {header[3]:<10} {header[4]:<10} {header[5]:<15}")  # 헤더 행 출력
print("-" * 65)  # 구분선 출력

# 정책 데이터를 반복하며 각 행을 형식에 맞춰 출력
for row_data in sarsa_q_policy_data:
    print(f"{str(row_data['State']):<10} {row_data['up']:<10.2f} {row_data['down']:<10.2f} {row_data['left']:<10.2f} {row_data['right']:<10.2f} {row_data['Optimal Action']:<15}")

**표 형식 SARSA Q-값 및 정책 분석**  

이 표는 각 상태-행동 쌍에 대한 학습된 Q-값과 각 상태에서 가장 높은 Q-값을 선택하여 결정된 최적 행동을 보여줍니다.  

**주요 관찰 사항:**  

1. **Q-값 추세 및 가치 전파:**  
   - **목표 (3,3)에 가까운 상태**는 더 나은 장기 보상 잠재력을 나타내므로 더 높은 Q-값을 갖습니다.  
   - 예를 들어, **상태 (3,2)는 오른쪽으로 이동하는 것에 대해 높은 Q-값 (10.00)**을 가지며, 이는 목표로 직접 이어진다는 것을 확인시켜 줍니다.  

2. **최적 행동 선택:**  
   - **최적 행동(Optimal Action)** 열은 학습된 정책이 주로 **오른쪽(→) 및 아래쪽(↓) 이동**을 선호하며 목표를 향한 논리적인 경로를 따른다는 것을 확인시켜 줍니다.  
   - 어떤 경우에는 대체 행동(예: 위쪽 또는 왼쪽)이 약간 낮은 Q-값을 가지며, 이는 훈련 중 **탐험 기반 조정**을 나타냅니다.  

3. **종료 상태:**  
   - 예상대로 **시작 상태 (0,0)와 목표 상태 (3,3)는 종료 상태**이며, 더 이상 행동이 취해지지 않습니다.  
   - 이들의 Q-값은 추가 학습에 기여하지 않으므로 **0**으로 유지됩니다.  

4. **절벽 회피 및 위험 인식 행동 (절벽 보행 예제 관련 - 이 표는 그리드 월드용):**  
   - (이 특정 표는 그리드 월드 예제에 대한 것이므로 절벽 회피는 직접적으로 관찰되지 않습니다. 절벽 보행 분석에서 더 관련성이 높습니다.)
   - SARSA의 **온-폴리시(on-policy)** 특성은 종종 Q-러닝에 비해 더 보수적인 전략으로 이어질 수 있으며, 이는 위험한 지역 근처에서 특히 두드러질 수 있습니다.  

5. **시각화와의 일관성:**  
   - 학습된 **정책은 화살표 그리드 시각화와 일치**하며, 에이전트가 예상 궤적을 따른다는 것을 확인시켜 줍니다.  
   - 약간의 불일치는 여러 행동이 유사한 기대 수익을 갖는 **Q-값 동점 처리** 때문일 수 있습니다.  

**Q-러닝과의 비교 (암묵적 차이점):**  
- **SARSA의 온-폴리시 업데이트**는 더 안전한 경로를 우선시하는 경향이 있으며, 이는 절벽 근처의 위험한 이동을 피하는 경우에 나타날 수 있습니다.  
- 반대로, **Q-러닝 (오프-폴리시)은 최대 보상을 위해 최적화하면서 더 자주 떨어질 위험을 감수하는 더 공격적인 정책**을 초래할 수 있습니다.  
- 차이점은 특히 **절벽 인접 상태**에서 두드러지며, SARSA는 더 **위험 회피적인 접근 방식**을 학습했을 가능성이 높습니다.

# 다른 하이퍼파라미터로 테스트 (선택 사항)

학습률 (α), 할인율 (γ), 초기 엡실론 (ε₀)에 대해 다른 값을 실험하여 SARSA의 학습 속도와 수렴에 미치는 영향을 관찰합니다.

In [ ]:
# SARSA에 대한 다른 하이퍼파라미터로 실험
learning_rates = [0.1, 0.5]        # 다른 학습률 테스트 (alpha = 0.1 및 0.5)
discount_factors = [0.9, 0.99]     # 다른 할인율 테스트 (gamma = 0.9 및 0.99)
exploration_rates = [1.0]          # 이 테스트에서는 초기 탐험률 (epsilon)을 1.0으로 유지
min_epsilon = 0.1                  # 최소 탐험률 (epsilon)
decay_rate = 0.01                  # 엡실론 감쇠율
episodes = 500                     # 훈련할 에피소드 수
max_steps = 100                    # 에피소드당 최대 단계 수

# 비교를 위해 결과 저장
sarsa_results = []

# 환경 파라미터 다시 정의
rows, cols = 4, 4                  # 그리드 차원 (4x4)
terminal_states = [(0, 0), (3, 3)] # 그리드의 종료 상태
rewards = {(0, 0): 1, (3, 3): 10}  # 종료 상태에 대한 보상
grid, state_space, action_space = create_gridworld(rows, cols, terminal_states, rewards)

# 다른 하이퍼파라미터 조합으로 실험 실행
print("Running SARSA with different hyperparameters...") # 다른 하이퍼파라미터로 SARSA 실행 중...
for alpha in learning_rates:  # 학습률 반복
    for gamma in discount_factors:  # 할인율 반복
        for initial_epsilon in exploration_rates:  # 초기 탐험률 반복
            print(f"  Training with alpha={alpha}, gamma={gamma}, epsilon_init={initial_epsilon}") # alpha={alpha}, gamma={gamma}, epsilon_init={initial_epsilon}로 훈련 중
            
            # 현재 하이퍼파라미터 세트로 SARSA 실행
            q_table, rewards_per_episode, episode_lengths = run_sarsa(
                state_space, action_space, rewards, terminal_states, rows, cols, alpha, gamma,
                initial_epsilon, min_epsilon, decay_rate, episodes, max_steps
            )

            # 나중 분석을 위해 결과 저장
            sarsa_results.append({
                'alpha': alpha,                          # 학습률
                'gamma': gamma,                          # 할인율
                'initial_epsilon': initial_epsilon,      # 초기 탐험률
                'rewards_per_episode': rewards_per_episode,  # 에피소드별 보상
                'episode_lengths': episode_lengths       # 에피소드 길이
            })
print("Experiments finished.") # 실험 완료.

# --- 하이퍼파라미터 효과 시각화 ---
num_results = len(sarsa_results)  # 테스트된 총 하이퍼파라미터 조합 수
# 서브플롯의 그리드 크기 결정 (예: 4개 결과의 경우 2x2)
plot_rows = int(np.ceil(np.sqrt(num_results)))  # 서브플롯 그리드의 행 수
plot_cols = int(np.ceil(num_results / plot_rows))  # 서브플롯 그리드의 열 수

# 플롯을 위한 그림 생성
plt.figure(figsize=(6 * plot_cols, 4 * plot_rows))  # 그리드 차원을 기반으로 그림 크기 조정

# 모든 하이퍼파라미터 조합을 시각화하기 위해 더 큰 그림 생성
plt.figure(figsize=(20, 5))

# 각 하이퍼파라미터 조합에 대한 에피소드별 보상 플로팅
for i, result in enumerate(sarsa_results):
    plt.subplot(plot_rows, plot_cols, i + 1)  # 각 결과에 대한 서브플롯 생성
    plt.plot(result['rewards_per_episode'])  # 에피소드별 보상 플로팅
    plt.title(f"SARSA: α={result['alpha']}, γ={result['gamma']}, ε₀={result['initial_epsilon']}")  # 하이퍼파라미터가 포함된 제목
    plt.xlabel('Episode')  # x축 레이블
    plt.ylabel('Total Reward')  # y축 레이블
    plt.grid(True)  # 가독성을 위해 그리드 추가
    # 더 나은 비교를 위해 모든 플롯에서 일관된 Y축 제한 설정
    plt.ylim(
        min(min(r['rewards_per_episode']) for r in sarsa_results) - 1, 
        max(max(r['rewards_per_episode']) for r in sarsa_results) + 1
    )

# 전체 그림에 대한 상위 제목 추가
plt.suptitle("SARSA Performance with Different Hyperparameters", fontsize=16, y=1.02) # 다른 하이퍼파라미터에 따른 SARSA 성능
plt.tight_layout()  # 겹침 방지를 위해 레이아웃 조정
plt.show()  # 플롯 표시

# 다른 환경에 SARSA 적용 (절벽 보행)
이 섹션에서는 SARSA 알고리즘을 다른 환경인 절벽 보행(Cliff Walking)에 적용할 것입니다. 이 환경은 에이전트가 목표 상태에 도달하기 위해 절벽(음의 보상)을 피하면서 그리드를 탐색해야 하는 고전적인 강화 학습 문제입니다.

In [ ]:
# 절벽 보행 환경 파라미터 정의
cliff_rows, cliff_cols = 4, 12
cliff_start_state = (3, 0)
cliff_terminal_state = (3, 11)
cliff_states = [(3, c) for c in range(1, 11)] # 절벽 상태들

# 보상 정의: 일반 걸음은 -1, 절벽은 -100, 목표는 +10 (목표 보상은 업데이트에서 암묵적으로 처리됨)
# SARSA/Q-러닝은 일반적으로 상태로 *전환*하는 것에 대한 보상을 사용합니다.
# 표준 절벽 보행 보상을 시뮬레이션할 수 있습니다: 걸음당 -1, 떨어지면 -100.
# 이 일반적인 설정을 위해 보상/전환 로직을 약간 수정해 봅시다.

# 표준 절벽 보행을 위한 수정된 상태 전이 및 보상
def cliff_state_transition_reward(
    state: Tuple[int, int],
    action: str,
    rows: int,
    cols: int,
    cliff_states: List[Tuple[int, int]],
    start_state: Tuple[int, int]
) -> Tuple[Tuple[int, int], int]:
    """
    절벽 보행에 대한 다음 상태와 보상을 계산합니다.
    보상은 일반 걸음은 -1, 절벽으로 떨어지면 -100입니다.
    절벽으로 떨어지면 상태를 start_state로 재설정합니다.
    """
    row, col = state
    next_row, next_col = row, col

    # 잠재적 다음 위치 계산
    if action == 'up' and row > 0:
        next_row -= 1
    elif action == 'down' and row < rows - 1:
        next_row += 1
    elif action == 'left' and col > 0:
        next_col -= 1
    elif action == 'right' and col < cols - 1:
        next_col += 1

    next_state = (next_row, next_col)

    # 보상 결정
    if next_state in cliff_states:
        reward = -100
        next_state = start_state # 절벽에 빠지면 시작 상태로 재설정
    elif next_state == cliff_terminal_state:
        reward = 0 # 표준 설정은 종종 목표 도달 시 0을 제공, -1 전환 비용은 단계 보상으로 처리됨
                   # 대안: 원한다면 여기에 +10 또는 다른 양수 보상을 제공. -1 단계 비용을 고수합시다.
        reward = -1 # 일관성을 위해 목표 도달 시에도 단계 비용 적용? 아니요, 목표 전환은 특별함.
                    # 일반적인 -1 단계 비용 구조를 사용합시다. 목표 상태 자체는 도착 시 보상/패널티 없음.
        reward = -1
    else:
        reward = -1 # 표준 단계 비용

    return next_state, reward

In [ ]:
# 절벽 보행 보상 구조를 위한 수정된 SARSA 에피소드 실행기
def run_sarsa_cliff_episode(
    q_table: Dict[Tuple[int, int], Dict[str, float]],
    action_space: List[str],
    terminal_state: Tuple[int, int],
    cliff_states: List[Tuple[int, int]],
    start_state: Tuple[int, int],
    rows: int,
    cols: int,
    alpha: float,
    gamma: float,
    epsilon: float,
    max_steps: int
) -> Tuple[int, int]:
    """ 절벽 보행을 위한 SARSA의 단일 에피소드를 실행합니다. """
    state = start_state
    action = epsilon_greedy_policy(state, q_table, action_space, epsilon)
    total_reward = 0
    steps = 0

    for _ in range(max_steps):
        next_state, reward = cliff_state_transition_reward(
            state, action, rows, cols, cliff_states, start_state
        )
        total_reward += reward

        next_action = epsilon_greedy_policy(next_state, q_table, action_space, epsilon)

        # (s, a)에 대한 Q-값 업데이트
        # 종료 상태 특별 처리: Q(종료 상태, 모든 행동) = 0
        if next_state == terminal_state:
             q_next = 0.0
        else:
             # q_table 접근 전에 경계 및 존재 확인
             if next_state in q_table and next_action in q_table[next_state]:
                 q_next = q_table[next_state][next_action]
             else: # 전체 초기화에서는 발생하지 않아야 하지만 안전 확인
                 q_next = 0.0

        td_target = reward + gamma * q_next
        td_error = td_target - (q_table[state][action] if state in q_table and action in q_table[state] else 0.0)
        if state in q_table and action in q_table[state]: # 업데이트 전에 상태-행동이 존재하는지 확인
             q_table[state][action] += alpha * td_error

        state = next_state
        action = next_action
        steps += 1

        if state == terminal_state:
            break

    return total_reward, steps

In [ ]:
# 절벽 보행을 위한 수정된 SARSA 실행 함수
def run_sarsa_cliff(
    action_space: List[str],
    terminal_state: Tuple[int, int],
    cliff_states: List[Tuple[int, int]],
    start_state: Tuple[int, int],
    rows: int,
    cols: int,
    alpha: float,
    gamma: float,
    initial_epsilon: float,
    min_epsilon: float,
    decay_rate: float,
    episodes: int,
    max_steps: int
) -> Tuple[Dict[Tuple[int, int], Dict[str, float]], List[int], List[int]]:
    """ 절벽 보행을 위해 SARSA를 실행합니다. """
    # 상태 공간 동적 생성 또는 전달 - 여기서는 내부에서 생성
    state_space = [(r, c) for r in range(rows) for c in range(cols)]
    q_table = initialize_q_table_nested(state_space, action_space)

    rewards_per_episode = []
    episode_lengths = []

    for episode in range(episodes):
        epsilon = adjust_epsilon(initial_epsilon, min_epsilon, decay_rate, episode)
        total_reward, steps = run_sarsa_cliff_episode(
            q_table, action_space, terminal_state, cliff_states, start_state,
            rows, cols, alpha, gamma, epsilon, max_steps
        )
        rewards_per_episode.append(total_reward)
        episode_lengths.append(steps)
        # 선택 사항: 진행 상황 출력
        # if (episode + 1) % 100 == 0:
        #     print(f"Episode {episode + 1}/{episodes} completed. Avg Reward (last 100): {np.mean(rewards_per_episode[-100:]):.2f}")


    return q_table, rewards_per_episode, episode_lengths

절벽 보행에 대해 SARSA를 실행해 봅시다.

In [ ]:
# --- 절벽 보행에서 SARSA 실행 ---
# 하이퍼파라미터 (튜닝 가능)
alpha_cliff = 0.1         # 학습률
gamma_cliff = 0.99        # 할인율 (높은 감마는 더 멀리 내다보도록 장려)
initial_epsilon_cliff = 0.2 # 1.0보다 적은 탐험으로 시작? 처음에는 0.2, 그 다음엔 1.0 시도
min_epsilon_cliff = 0.01   # 더 낮은 최소 엡실론
decay_rate_cliff = 0.005   # 더 느린 감쇠?
episodes_cliff = 500       # 에피소드 수
max_steps_cliff = 200      # 에피소드당 최대 단계 수

print("Running SARSA on Cliff Walking environment...") # 절벽 보행 환경에서 SARSA 실행 중...
# 행동 공간 정의
cliff_action_space = ['up', 'down', 'left', 'right']

# 훈련 실행
cliff_q_table_sarsa, cliff_rewards_sarsa, cliff_lengths_sarsa = run_sarsa_cliff(
    cliff_action_space, cliff_terminal_state, cliff_states, cliff_start_state,
    cliff_rows, cliff_cols, alpha_cliff, gamma_cliff,
    initial_epsilon_cliff, min_epsilon_cliff, decay_rate_cliff, episodes_cliff, max_steps_cliff
)
print("SARSA training on Cliff Walking finished.") # 절벽 보행 SARSA 훈련 완료.

보상을 플로팅하려면 다음 코드를 사용할 수 있습니다.

In [ ]:
# 절벽 보행 환경에 대한 보상 플로팅
def plot_rewards(rewards_per_episode: List[int], ax: plt.Axes = None) -> plt.Axes:
    """
    에피소드에 걸쳐 누적된 총 보상을 플로팅합니다.

    매개변수:
    - rewards_per_episode (List[int]): 에피소드별 총 보상 목록.
    - ax (plt.Axes, optional): 플로팅할 Matplotlib 축. None이면 새 그림과 축이 생성됩니다.

    반환값:
    - plt.Axes: 플롯이 포함된 Matplotlib 축.
    """
    # 축이 제공되지 않으면 새 그림과 축 생성
    if ax is None:
        fig, ax = plt.subplots(figsize=(8, 6))
    
    # 에피소드에 걸쳐 보상 플로팅
    ax.plot(rewards_per_episode)
    ax.set_xlabel('Episode')  # x축 레이블
    ax.set_ylabel('Total Reward')  # y축 레이블
    ax.set_title('Rewards Over Episodes')  # 플롯 제목
    
    # 필요한 경우 추가 사용자 정의를 위해 축 반환
    return ax

결과를 시각화하기 위해 이전과 유사하게 보상과 에피소드 길이를 플로팅할 수 있습니다. 절벽 보행 환경에 대한 학습된 정책도 시각화해 봅시다.

In [ ]:
# --- 절벽 보행 시각화 (SARSA) ---
# 여러 플롯을 위한 그림 생성
fig_cliff, axs_cliff = plt.subplots(2, 2, figsize=(18, 8)) # 크기 약간 조정

# 1. 보상 플로팅
plot_rewards(cliff_rewards_sarsa, ax=axs_cliff[0, 0])
axs_cliff[0, 0].set_title("SARSA: Cliff Walking Rewards") # SARSA: 절벽 보행 보상
axs_cliff[0, 0].grid(True)

# 2. 에피소드 길이 플로팅
axs_cliff[0, 1].plot(cliff_lengths_sarsa)
axs_cliff[0, 1].set_xlabel('Episode')
axs_cliff[0, 1].set_ylabel('Episode Length')
axs_cliff[0, 1].set_title('SARSA: Cliff Walking Episode Lengths') # SARSA: 절벽 보행 에피소드 길이
axs_cliff[0, 1].grid(True)

# 최대 Q-값 플로팅 (히트맵) - 절벽 환경 매개변수를 받는 함수 필요
def plot_q_values_cliff(q_table, rows, cols, ax):
    q_values = np.zeros((rows, cols))
    for r in range(rows):
        for c in range(cols):
            state = (r, c)
            if state in q_table and q_table[state]:
                q_values[r, c] = max(q_table[state].values())
            else:
                q_values[r,c] = -np.inf # 방문 안 함/종료 상태 표시

    # 더 나은 시각화를 위해 절벽 상태 마스크
    q_values_masked = np.ma.masked_where(q_values <= -100, q_values) # 절벽의 극단적인 음수 숨기기

    im = ax.imshow(q_values_masked, cmap='viridis')
    plt.colorbar(im, ax=ax, label='Max Q-Value')
    ax.set_title('SARSA: Max Q-Values (Cliff)') # SARSA: 최대 Q-값 (절벽)
    ax.set_xticks(np.arange(cols))
    ax.set_yticks(np.arange(rows))
    ax.set_xticklabels([])
    ax.set_yticklabels([])
    ax.grid(which='major', color='w', linestyle='-', linewidth=1)

plot_q_values_cliff(cliff_q_table_sarsa, cliff_rows, cliff_cols, ax=axs_cliff[1, 0])


# 4. 정책 플로팅
plot_policy_grid(cliff_q_table_sarsa, cliff_rows, cliff_cols, [cliff_terminal_state], ax=axs_cliff[1, 1]) # 종료 상태 전달
axs_cliff[1, 1].set_title("SARSA: Learned Policy (Cliff)") # SARSA: 학습된 정책 (절벽)


plt.tight_layout()
plt.show()

**주요 관찰 사항:**  

1. **왼쪽 위: 에피소드별 보상**  
   - 총 보상이 시간이 지남에 따라 **상당히 향상되어** 성공적인 학습을 나타냅니다.  
   - 초기 에피소드는 절벽으로 자주 떨어져 큰 음의 보상을 보이는 등 **높은 변동성**을 보입니다.  
   - 훈련이 진행됨에 따라 보상은 **0에 가까워지며 안정화**되어 에이전트가 최적에 가까운 정책을 학습했음을 시사합니다.  

2. **오른쪽 위: 시간에 따른 에피소드 길이**  
   - 처음에는 에피소드 길이가 **매우 길어(200단계 이상)** 에이전트가 목표에 효율적으로 도달하는 데 어려움을 겪고 있음을 의미합니다.  
   - 학습이 진행됨에 따라 에피소드 길이는 **급격히 감소**하여 약 200 에피소드 후 20-30 단계 정도로 안정화됩니다.  
   - 간헐적인 급증은 **탐험 기반의 이탈**을 나타내지만, 전반적으로 에이전트는 최적 경로로 수렴합니다.  

3. **왼쪽 아래: 상태별 최대 Q-값 (히트맵)**  
   - **절벽 지역에서 Q-값이 가장 낮으며**, 이는 에이전트가 위험을 인식하고 있음을 확인시켜 줍니다.  
   - 그리드의 **위쪽 영역은 Q-값이 더 높으며**, 더 안전하고 보람 있는 경로를 나타냅니다.  
   - **목표 상태는 Q-값이 가장 높으며**, 그 선호도를 강화합니다.  

4. **오른쪽 아래: 학습된 정책 (화살표 그리드)**  
   - **화살표는 각 상태에서 에이전트의 최적 행동**을 나타냅니다.  
   - 에이전트는 주로 **오른쪽(→) 및 아래쪽(↓) 전략**을 따르며, 목표로 가는 효율적인 경로를 우선시합니다.  
   - **절벽 인접 행**에서는 에이전트가 떨어지는 것을 피하기 위해 종종 **위쪽(↑) 또는 왼쪽(←)**으로 이동하는 등 더 신중한 접근 방식을 취합니다.  
   - 오른쪽 아래의 종료 상태("T")는 목표를 표시합니다.  

**분석:**  
- 에이전트는 탐험과 활용의 균형을 맞추며 **최적의 정책을 성공적으로 학습**합니다.  
- 초기 탐험은 **높은 실패율**로 이어지지만, SARSA는 에이전트가 더 안전한 결정을 향해 정책을 개선할 수 있도록 합니다.  
- **Q-값 분포는 위험 인식을 확인**시켜 주며, 보상을 최적화하면서 절벽을 피합니다.  
- **Q-러닝과 비교할 때, SARSA는 일반적으로 더 안전한 경로를 선호하며**, 이는 위험한 지역 근처에서 관찰된 정책 조정과 일치합니다.  

전반적으로 SARSA 에이전트는 **절벽 환경을 효율적으로 탐색**하여 페널티를 최소화하면서 성공률을 향상시키는 방법을 학습합니다.

## 일반적인 문제점 및 해결책

**문제점: 느린 학습 또는 막힘 현상**
*   **해결책**: 하이퍼파라미터(α, γ, ε 감쇠)를 조정합니다. 초기에 탐험을 늘리거나 더 정교한 탐험(예: 볼츠만 탐색)을 사용합니다. 매우 큰 상태 공간의 경우 함수 근사법이 필요할 수 있습니다(여기서는 사용되지 않음).

**문제점: 탐험과 활용의 균형**
*   **해결책**: 잘 조정된 엡실론 감쇠 스케줄을 사용합니다. 높은 값(1.0 근처)에서 시작하여 작은 값(예: 0.01 또는 0.1)으로 감쇠합니다. 감쇠율은 에이전트가 탐험에서 활용으로 얼마나 빨리 전환하는지 결정합니다.

**문제점: 적절한 하이퍼파라미터 선택**
*   **해결책**: 실험이 중요합니다. 일반적인 시작점:
    *   α (학습률): 0.1이 종종 좋은 시작점입니다. 높은 값은 더 빨리 학습하지만 불안정할 수 있습니다.
    *   γ (할인율): 0.9 ~ 0.99. 높은 값은 미래 보상을 더 강조합니다.
    *   ε (엡실론): 높은 값(1.0)에서 시작하여 낮은 값(0.1 또는 0.01)으로 감쇠합니다. 감쇠율을 조정합니다.

**문제점: 초기 Q-값에 대한 민감성**
*   **해결책**: Q-값을 낙관적으로 초기화(탐험 장려)하거나 0으로 초기화합니다. 일관성이 중요합니다.

## SARSA 대 다른 강화 학습 알고리즘

### SARSA의 장점
-   **온-폴리시(On-Policy):** 실행 중인 정책의 가치를 학습하므로 정책 평가나 현재 행동을 이해하는 것이 중요한 상황에서 유용할 수 있습니다.
-   **종종 더 안정적/보수적:** 업데이트 시 탐험 단계를 고려하기 때문에 Q-러닝에 비해 지나치게 낙관적이거나 위험한 정책을 학습하는 경향이 적을 수 있으며, 특히 위험 요소(예: 절벽) 근처에서 그렇습니다.
-   **수렴 보장:** Q-러닝과 유사한 조건 하에서 수렴합니다(모든 상태-행동 쌍이 무한히 자주 방문되고 학습률이 적절히 감쇠하는 경우).
-   비교적 이해하고 구현하기 쉽습니다.

### SARSA의 한계점
-   **더 느릴 수 있음:** 잠재적으로 차선책인 탐험적 행동을 기반으로 학습하기 때문에 최적 정책 학습이 Q-러닝보다 오래 걸릴 수 있습니다.
-   **학습 중 차선책 정책:** 현재 (아마도 탐험적인) 정책을 기반으로 학습하기 때문에 중간 단계에서 학습된 정책은 Q-러닝이 추정할 수 있는 것보다 덜 최적일 수 있습니다.
-   **동일한 상태 공간 한계:** Q-러닝과 마찬가지로 테이블 형식 SARSA는 매우 크거나 연속적인 상태/행동 공간에서 어려움을 겪습니다.

### 관련 알고리즘
-   **Q-러닝**: 고전적인 오프-폴리시 TD 제어 알고리즘입니다. 최적의 행동-가치 함수를 직접 학습합니다.
-   **기대 SARSA (Expected SARSA)**: 단일 샘플링된 다음 행동의 Q-값 대신 다음 상태의 *기대* Q-값(정책에 따른 행동에 대한 평균)을 사용하는 변형입니다. 종종 SARSA보다 성능이 좋습니다.
-   **SARSA(λ) / True Online SARSA(λ)**: 적격성 추적(eligibility traces)을 통합하여 상태 및 행동 시퀀스를 통해 업데이트를 역전파함으로써 학습 속도를 높입니다.
-   **액터-크리틱 방법 (Actor-Critic Methods)**: 가치 기반 접근 방식(SARSA/Q-러닝 등)과 정책 기반 접근 방식을 결합하여 가치 함수와 정책을 모두 명시적으로 학습합니다.

## 결론

SARSA는 강화 학습에서 기본적인 온-폴리시 시간차(TD) 제어 알고리즘입니다. 핵심 특징은 현재 정책(상태-행동-보상-상태-행동 5요소)에 따라 다음 상태에서 실제로 취해진 행동을 기반으로 Q-값을 업데이트하는 것입니다.

이러한 온-폴리시 특성은 특히 상당한 위험이 있는 환경에서 오프-폴리시 대응 알고리즘인 Q-러닝에 비해 더 보수적이지만 잠재적으로 더 안정적인 학습으로 이어지는 경우가 많습니다. 절벽 보행 예제에서 보여주듯이 SARSA는 더 안전한 경로를 학습하는 경향이 있습니다. 테이블 형식 SARSA는 Q-러닝과 동일하게 큰 상태 공간에 대한 한계를 공유하지만, 그 원칙은 복잡한 영역에서 사용되는 더 발전된 온-폴리시 알고리즘의 기초를 형성합니다. SARSA를 이해하는 것은 온-폴리시 학습 및 TD 제어의 핵심 개념에 대한 귀중한 통찰력을 제공합니다.